In [4]:
import os ; import numpy as np ; import pandas as pd ; import matplotlib.pyplot as plt ; import pyemu ; plt.rcParams['font.size'] = 12
from pathlib import Path; print(sys.version) ; import sys; import flopy
print(f"numpy version: {np.__version__}") ; print(f"flopy version: {flopy.__version__}")

3.11.11 | packaged by conda-forge | (main, Dec  5 2024, 14:06:23) [MSC v.1942 64 bit (AMD64)]
numpy version: 1.26.4
flopy version: 3.8.2


In [ ]:
sim_ws1   = tmp_d = Path('E:\\15_REPOS\\00_BETAMI\\97_GMDSI\\00\\part2_01b')  #; sim_name = "freyberg_mf6"
sim_ws2   = Path("./part2_01b2")                                                                        ; sim_ws2.mkdir(exist_ok=True) 
sim_ws3   = Path("./part2_01b2/beta")                                                                   ; sim_ws3.mkdir(exist_ok=True)
shutil.copytree(sim_ws1, sim_ws2, dirs_exist_ok=True)      #;sim=flopy.mf6.MFSimulation.load(sim_ws=sim_ws2,verbosity_level=0) ;gwf = sim.get_model()

In [ ]:
ID = "freyberg6_template"
assert os.path.exists(ID)," need to run the setup_pestpp_interface notebook first!"

In [ ]:
pst = pyemu.Pst(os.path.join(ID,"freyberg6.pst"))

In [ ]:
pst.pestpp_options

In [ ]:
pst.control_data.noptmax = -1
pst_filename = "freyberg6_pmc.pst"
pst.write(os.path.join(ID,pst_filename),version=2)

In [ ]:
pyemu.os_utils.start_workers(ID,"pestpp-ies",pst_filename,num_workers=8,master_dir="master_pmc", worker_root='.')

In [ ]:
obs_filename = pst_filename.replace(".pst",".0.obs.csv")
obs_df = pd.read_csv(os.path.join("master_pmc",obs_filename),index_col=0)

In [ ]:
obs_df

In [ ]:
pst.obs_groups

In [ ]:
pst.try_parse_name_metadata()
#pst.observation_data.loc[:,"time"] = pst.observation_data.time.apply(np.float)
obs = pst.observation_data
for obs_group in pst.obs_groups:
    obs_g = obs.loc[obs.obgnme==obs_group,:].copy()
    obs_g.sort_values(by="datetime",inplace=True)
    obs_g_df = obs_df.loc[:,obs_g.obsnme]
    fig,ax = plt.subplots(1,1,figsize=(15,2))
    [ax.plot(obs_g.datetime,obs_g_df.loc[i,obs_g.obsnme],color='0.5',alpha=0.5,lw=0.2) for i in obs_g_df.index]
    ax.set_title(obs_group)
    plt.setp( ax.xaxis.get_majorticklabels(), rotation=45, horizontalalignment='right' )
    plt.show()
    
    